<h1><center> Limpieza de datos</h1>

# Librerias

In [2]:
import pandas as pd

# Archivos

In [6]:
screen_times = pd.read_csv("dataset/screentime_analysis.csv")
moblie_usuage = pd.read_csv("dataset/mobile_usage_behavioral_analysis.csv")
google_play_store_user = pd.read_csv("dataset/google/googleplaystore_user_reviews.csv")
google_play_store = pd.read_csv("dataset/google/googleplaystore.csv")
mbr = pd.read_csv("dataset/mobilerec_part_1.csv")

## **screentime_analysis**


In [244]:
original_len = len(screen_times)
screen_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             200 non-null    object
 1   App              200 non-null    object
 2   Usage (minutes)  200 non-null    int64 
 3   Notifications    200 non-null    int64 
 4   Times Opened     200 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.9+ KB


In [245]:
screen_times = screen_times.drop_duplicates()
screen_times.columns = screen_times.columns.str.strip()

In [246]:
# Convertir la columna 'Date' a tipo datetime
screen_times['Date'] = pd.to_datetime(screen_times['Date'], errors='coerce')
screen_times = screen_times.dropna(subset=['Date'])

In [247]:
screen_times.reset_index(drop=True, inplace=True)

In [248]:
# Ejemplo: screen_times = screen_times.drop_duplicates()
final_len = len(screen_times)
print(f"screen_times: Se eliminaron {original_len - final_len} filas.")


screen_times: Se eliminaron 0 filas.


## **moblie_usuage_behavioral_analysis**

In [249]:
moblie_usuage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   User_ID                       1000 non-null   int64  
 1   Age                           1000 non-null   int64  
 2   Gender                        1000 non-null   object 
 3   Total_App_Usage_Hours         1000 non-null   float64
 4   Daily_Screen_Time_Hours       1000 non-null   float64
 5   Number_of_Apps_Used           1000 non-null   int64  
 6   Social_Media_Usage_Hours      1000 non-null   float64
 7   Productivity_App_Usage_Hours  1000 non-null   float64
 8   Gaming_App_Usage_Hours        1000 non-null   float64
 9   Location                      1000 non-null   object 
dtypes: float64(5), int64(3), object(2)
memory usage: 78.3+ KB


In [250]:
moblie_usuage = moblie_usuage.drop_duplicates()
moblie_usuage.columns = moblie_usuage.columns.str.strip()

moblie_usuage['Gender'] = moblie_usuage['Gender'].str.strip()
moblie_usuage['Location'] = moblie_usuage['Location'].str.strip()

In [251]:
moblie_usuage.reset_index(drop=True, inplace=True)

## **gogleplay**

### googleplaystore_user_reviews.csv

In [252]:
google_play_store_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   App                     64295 non-null  object 
 1   Translated_Review       37427 non-null  object 
 2   Sentiment               37432 non-null  object 
 3   Sentiment_Polarity      37432 non-null  float64
 4   Sentiment_Subjectivity  37432 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [253]:
google_play_store_user = google_play_store_user.drop_duplicates() # se eliminan muchoo
google_play_store_user.columns = google_play_store_user.columns.str.strip()

In [254]:
# No se que tan necesario son estas columnas
google_play_store_user = google_play_store_user.dropna(subset=[ # se eliminan mucho
    'Translated_Review', 'Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity'
])

In [255]:
google_play_store_user.reset_index(drop=True, inplace=True)

In [256]:
original_len = 64295
final_len = len(google_play_store_user)
print(f"Se eliminaron {original_len - final_len} filas incompletas.")

Se eliminaron 34603 filas incompletas.


### googleplaystore.csv

In [257]:
google_play_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [258]:
google_play_store = google_play_store.drop_duplicates()
google_play_store.columns = google_play_store.columns.str.strip()

google_play_store = google_play_store.dropna(subset=['Rating', 'Type', 'Content Rating']) # se eliminan muchas filas

google_play_store['Reviews'] = pd.to_numeric(google_play_store['Reviews'], errors='coerce')

In [259]:
# Size
# Cambiar "M, k y Varies with device" a numero
def parse_size(size):
    if size.endswith('M'):
        return float(size.replace('M', '')) * 1_000_000
    elif size.endswith('k'):
        return float(size.replace('k', '')) * 1_000
    elif size == 'Varies with device':
        return None
    else:
        return None

google_play_store['Size'] = google_play_store['Size'].apply(parse_size)

In [260]:
# Installs
google_play_store['Installs'] = (
    google_play_store['Installs']
    .str.replace('[+,]', '', regex=True)
    .str.strip()
)
google_play_store['Installs'] = pd.to_numeric(google_play_store['Installs'], errors='coerce')

In [261]:
# Price
google_play_store['Price'] = (
    google_play_store['Price']
    .str.replace('$', '', regex=False)
)
google_play_store['Price'] = pd.to_numeric(google_play_store['Price'], errors='coerce')

In [262]:
# 9. Convertir la columna a'Last Updated' a tipo datetime
google_play_store['Last Updated'] = pd.to_datetime(google_play_store['Last Updated'], errors='coerce')

In [263]:
google_play_store.reset_index(drop=True, inplace=True)

# MobileRec

In [15]:
mbr.head()

,app_package,review,rating,votes,date,uid,formated_date,unix_timestamp,app_category
0,com.cleverapps.heroes,It's really a fun game,5,1,"October 21, 2018",shqoc6X1fcJRLEmx,2018-10-21,1.540094e+09,Casual
1,com.bodyfast,uninstalling. it was ok but felt like it was c...,2,0,"January 18, 2019",shqoc6X1fcJRLEmx,2019-01-18,1.547788e+09,Health & Fitness
2,com.thrivegames.wordshapes,Love this game,4,1,"January 16, 2021",shqoc6X1fcJRLEmx,2021-01-16,1.610773e+09,Word
3,com.affinity.rewarded_play,Doesn't update play time. Just downloaded it 3...,1,1,"November 4, 2021",shqoc6X1fcJRLEmx,2021-11-04,1.635998e+09,Entertainment
4,dating.inmessage.net,app crashes every time I try to log in. what g...,1,0,"November 24, 2021",shqoc6X1fcJRLEmx,2021-11-24,1.637730e+09,Dating


In [9]:
print(mbr.shape)
mbr.info()

(100000, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   app_package     100000 non-null  object 
 1   review          99993 non-null   object 
 2   rating          100000 non-null  int64  
 3   votes           100000 non-null  int64  
 4   date            100000 non-null  object 
 5   uid             100000 non-null  object 
 6   formated_date   100000 non-null  object 
 7   unix_timestamp  100000 non-null  float64
 8   app_category    100000 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 6.9+ MB


In [ ]:
mbr["review"].isnull().sum() 
# Eliminar filas con reseñas nulas
mbr = mbr.dropna(subset=["review"])


# Guardar archivos

In [264]:
screen_times.to_csv('archivos_limpios/screen_times_limpio.csv', index=False)
moblie_usuage.to_csv('archivos_limpios/moblie_usuage_limpio.csv', index=False)
google_play_store_user.to_csv('archivos_limpios/google_play_store_user_limpio.csv', index=False)
google_play_store.to_csv('archivos_limpios/google_play_store_limpio.csv', index=False)